# Yiming's POP variance budget

1. Compress this dataset

In [1]:
import xarray as xr

file = "/glade/scratch/yguo20/T_variance_budget_3way_balance_JRA55run_Yr42_Yr61_mean.nc"
ds = xr.open_dataset(file, chunks={"lat": 400, "lon": 800})
ds

<xarray.Dataset>
Dimensions:  (lat: 2400, lon: 3600, depth: 62)
Dimensions without coordinates: lat, lon, depth
Data variables:
    TLAT     (lat, lon) float64 dask.array<chunksize=(400, 800), meta=np.ndarray>
    TLONG    (lat, lon) float64 dask.array<chunksize=(400, 800), meta=np.ndarray>
    z_t      (depth) float64 dask.array<chunksize=(62,), meta=np.ndarray>
    BC       (depth, lat, lon) float64 dask.array<chunksize=(62, 400, 800), meta=np.ndarray>
    PKC      (depth, lat, lon) float64 dask.array<chunksize=(62, 400, 800), meta=np.ndarray>
    VMIX     (depth, lat, lon) float64 dask.array<chunksize=(62, 400, 800), meta=np.ndarray>
    HDIFF    (depth, lat, lon) float64 dask.array<chunksize=(62, 400, 800), meta=np.ndarray>

In [2]:
newds = ds.rename({"z_t": "depth"}).set_coords(["depth", "TLAT", "TLONG"])
newds.TLAT.attrs = {"standard_name": "latitude", "units": "degrees_north"}
newds.TLONG.attrs = {"standard_name": "longitude", "units": "degrees_east"}
newds["depth"] = newds.depth / 100
newds.depth.attrs = {"standard_name": "depth", "units": "m", "positive": "down"}
newds

<xarray.Dataset>
Dimensions:  (lat: 2400, lon: 3600, depth: 62)
Coordinates:
    TLAT     (lat, lon) float64 dask.array<chunksize=(400, 800), meta=np.ndarray>
    TLONG    (lat, lon) float64 dask.array<chunksize=(400, 800), meta=np.ndarray>
  * depth    (depth) float64 0.05 0.15 0.25 0.35 ... 51.25 53.75 56.25 58.75
Dimensions without coordinates: lat, lon
Data variables:
    BC       (depth, lat, lon) float64 dask.array<chunksize=(62, 400, 800), meta=np.ndarray>
    PKC      (depth, lat, lon) float64 dask.array<chunksize=(62, 400, 800), meta=np.ndarray>
    VMIX     (depth, lat, lon) float64 dask.array<chunksize=(62, 400, 800), meta=np.ndarray>
    HDIFF    (depth, lat, lon) float64 dask.array<chunksize=(62, 400, 800), meta=np.ndarray>

In [3]:
import distributed

client = distributed.Client(n_workers=4, threads_per_worker=1)

In [6]:
client.dashboard_url

AttributeError: 'Client' object has no attribute 'dashboard_url'

In [4]:
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: https://https://jupyterhub.hpc.ucar.edu/stable/user/dcherian/proxy/{port}/status,
Status: running,Using processes: True
Dashboard: https://https://jupyterhub.hpc.ucar.edu/stable/user/dcherian/proxy/{port}/status,Workers: 4
Total threads: 4,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:44679,Workers: 4
Dashboard: https://https://jupyterhub.hpc.ucar.edu/stable/user/dcherian/proxy/{port}/status,Total threads: 4
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:33681,Total threads: 1
Dashboard: https://https://jupyterhub.hpc.ucar.edu/stable/user/dcherian/proxy/{port}/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:37723,


In [5]:
import zarr

compressor = zarr.Blosc(cname="zstd", clevel=3, shuffle=2)
encoding = {"compressor": compressor}
newds.to_zarr(
    "pop-variance-budget.zarr", mode="w", encoding={var: encoding for var in newds}
)